basic

In [4]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# reference_image = cv2.imread(r'dataset/dhruv_reference.jpg')
reference_image = cv2.imread(r'dataset/amit_advanced.jpg')
threshold = 0.1
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

reference_results = pose.process(cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB))

reference_landmarks = []
if reference_results.pose_landmarks:
    for i, landmark in enumerate(reference_results.pose_landmarks.landmark):
        reference_landmarks.append((landmark.x, landmark.y, i))

# input_video_path = r'dataset/videos/pcc.mp4'
input_video_path = r'dataset/test.mp4'
# input_video = cv2.VideoCapture(0)
input_video = cv2.VideoCapture(input_video_path)
output_video_path = 'advancedvriksh/overallmix.mp4'  # Choose your desired output file name
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(input_video.get(cv2.CAP_PROP_FPS))
frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object to save the output video.
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
keypoint_labels = {
    "Perfect": "Perfect! Keep Going :)",
    "HandsNotAtRightPosition": "Hands not at right position",
    "HandsNotAt90": "Hands not at 90 degree",
    "LegsNotTriangle": "Legs not triangle",
    "Hands_legs_wrong": "Hands and leg both wrong",
    "LegDown": "Leg Down",
    "Idle": "Idle, please perform asana",
    "BentRight": "Bent right, \nstraighten yourself",
    "BentLeft": "Bent left, \nstraighten yourself",
    "BentForward": "Bent forward, straighten yourself",
    "WrongPosture" : "Straighten yourself"
}


while True:
    ret, frame = input_video.read()

    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_results = pose.process(frame_rgb)

    frame_landmarks = []
    if frame_results.pose_landmarks:
        for i, landmark in enumerate(frame_results.pose_landmarks.landmark):
            frame_landmarks.append((landmark.x, landmark.y, i))

    if len(frame_landmarks) > 0 and len(reference_landmarks) > 0:
        initial_offset = np.array(reference_landmarks[0][:2]) - np.array(frame_landmarks[0][:2])

    frame_copy = frame.copy()
    leg_down = False
    hands_not_at_right_position = False
    hands_not_at_90 = False
    bent_right = False
    bent_left = False
    bent_forward = False
    LegsNotTriangle = False

    #DIFFERENT
    right_shoulder_y = None
    left_shoulder_y = None


    for frame_landmark in frame_landmarks:
        frame_point = frame_landmark[:2]
        frame_keypoint_number = frame_landmark[2]

        if frame_keypoint_number == 11:  # Left Shoulder
            left_shoulder_y = frame_point[1]
        elif frame_keypoint_number == 12:  # Right Shoulder
            right_shoulder_y = frame_point[1]
        elif frame_keypoint_number == 13:  # Left Elbow
            left_elbow_y = frame_point[1]
        elif frame_keypoint_number == 14:  # Right Elbow
            right_elbow_y = frame_point[1]
        
        adjusted_left_shoulder_point = (reference_landmarks[11][0] - initial_offset[0], reference_landmarks[11][1] - initial_offset[1])
        adjusted_right_shoulder_point = (reference_landmarks[12][0] - initial_offset[0], reference_landmarks[12][1] - initial_offset[1])

        # Obtain y-coordinates of the adjusted reference frame for left and right shoulders
        adjusted_left_shoulder_y = adjusted_left_shoulder_point[1]
        adjusted_right_shoulder_y = adjusted_right_shoulder_point[1]

        adjusted_reference_point = (reference_landmarks[frame_keypoint_number][0] - initial_offset[0], reference_landmarks[frame_keypoint_number][1] - initial_offset[1])
        distance = calculate_distance(adjusted_reference_point, frame_point)
        

        
        if distance < threshold:
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 255, 0), -1)
        else:
            
            if frame_keypoint_number in [28, 30, 32] or frame_keypoint_number in [27, 29, 31] or frame_keypoint_number == 25 or frame_keypoint_number == 26:
                leg_down = True
            if frame_keypoint_number == 13 or frame_keypoint_number == 14:
                hands_not_at_90 = True
            if frame_keypoint_number in [16, 18, 20, 22] or frame_keypoint_number in [15, 17, 19, 21]:
                hands_not_at_right_position = True
            if frame_keypoint_number == 11:
                bent_left = True
            if frame_keypoint_number == 12:
                bent_right = True
                
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 0, 255), -1)

    if bent_left :
            label = keypoint_labels["BentLeft"]
    elif bent_right :
        label = keypoint_labels["BentRight"]
    elif leg_down:
        if hands_not_at_right_position:
            if not hands_not_at_90:
                label = keypoint_labels["Idle"]
            else:
                label = keypoint_labels["Hands_legs_wrong"]
        else:
            label = keypoint_labels["LegDown"]
    elif hands_not_at_90:
        label = keypoint_labels["HandsNotAt90"]
    elif hands_not_at_right_position:
        label = keypoint_labels["HandsNotAtRightPosition"]
    else:
        label = keypoint_labels["Perfect"]

    label_lines = label.split('\n')
    for i, line in enumerate(label_lines):
        cv2.putText(frame_copy, line, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    output_video.write(frame_copy)
    cv2.imshow("Processed Video", frame_copy)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

input_video.release()
output_video.release()
cv2.destroyAllWindows()


advanced - live

In [7]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# reference_image = cv2.imread(r'dataset/dhruv_reference.jpg')
reference_image = cv2.imread(r'dataset/amit_advanced.jpg')

pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

reference_results = pose.process(cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB))

reference_landmarks = []
if reference_results.pose_landmarks:
    for i, landmark in enumerate(reference_results.pose_landmarks.landmark):
        reference_landmarks.append((landmark.x, landmark.y, i))

# input_video_path = r'dataset/videos/pcd.mp4'
input_video_path = r'dataset/test_advanced.mp4'
input_video = cv2.VideoCapture(input_video_path)
# input_video = cv2.VideoCapture(input_video_path)
output_video_path = 'advancedvriksh/overallmix.mp4'  # Choose your desired output file name
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(input_video.get(cv2.CAP_PROP_FPS))
frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object to save the output video.
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
keypoint_labels = {
    "Perfect": "Perfect! \nKeep Going :)",
    "HandsNotAtRightPosition": "Hands not at right position",
    "HandsNotAbove": "Raise your \nhands over head", #elbow and eye
    "LegsInTriangle": "Raise your sole above your knee", #index foot and knee
    "Hands_legs_wrong": "Hands and leg both wrong",
    "LegDown": "Leg Down",
    "Idle": "Idle, please perform asana",
    "BentRight": "Bent right, \nstraighten yourself",
    "BentLeft": "Bent left, \nstraighten yourself",
    "BentForward": "Bent forward, straighten yourself"
}


while True:
    ret, frame = input_video.read()

    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_results = pose.process(frame_rgb)

    frame_landmarks = []
    if frame_results.pose_landmarks:
        for i, landmark in enumerate(frame_results.pose_landmarks.landmark):
            frame_landmarks.append((landmark.x, landmark.y, i))

    if len(frame_landmarks) > 0 and len(reference_landmarks) > 0:
        initial_offset = np.array(reference_landmarks[0][:2]) - np.array(frame_landmarks[0][:2])

    frame_copy = frame.copy()
    leg_down = False
    hands_not_at_right_position = False
    hands_not_above = False
    bent_right = False
    bent_left = False
    bent_forward = False
    LegsNotTriangle = False

    #DIFFERENT
    right_shoulder_y = None
    left_shoulder_y = None
    # left_elbow_y = None
    # right_elbow_y = None
    # left_eye_y = None
    # right_eye_y = None


    for frame_landmark in frame_landmarks:
        frame_point = frame_landmark[:2]
        frame_keypoint_number = frame_landmark[2]

        if frame_keypoint_number == 11:  # Left Shoulder
            left_shoulder_y = frame_point[1]
        if frame_keypoint_number == 12:  # Right Shoulder
            right_shoulder_y = frame_point[1]
        if frame_keypoint_number == 13:  # Left Elbow
            left_elbow_y = frame_point[1]
        if frame_keypoint_number == 14:  # Right Elbow
            right_elbow_y = frame_point[1]
        if frame_keypoint_number == 2:  # Left eye
            left_eye_y = frame_point[1]
        if frame_keypoint_number == 5:  # Right eye
            right_eye_y = frame_point[1]
        
        adjusted_left_shoulder_point = (reference_landmarks[11][0] - initial_offset[0], reference_landmarks[11][1] - initial_offset[1])
        adjusted_right_shoulder_point = (reference_landmarks[12][0] - initial_offset[0], reference_landmarks[12][1] - initial_offset[1])

        # Obtain y-coordinates of the adjusted reference frame for left and right shoulders
        adjusted_left_shoulder_y = adjusted_left_shoulder_point[1]
        adjusted_right_shoulder_y = adjusted_right_shoulder_point[1]

        adjusted_reference_point = (reference_landmarks[frame_keypoint_number][0] - initial_offset[0], reference_landmarks[frame_keypoint_number][1] - initial_offset[1])
        distance = calculate_distance(adjusted_reference_point, frame_point)
        threshold = 0.1

        
        if distance < threshold:
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 255, 0), -1)
        else:
            
            if frame_keypoint_number in [28, 30, 32] or frame_keypoint_number in [27, 29, 31] or frame_keypoint_number == 25 or frame_keypoint_number == 26:
                leg_down = True
            if frame_keypoint_number == 13 or frame_keypoint_number == 14:        
                hands_not_above = True
            if frame_keypoint_number in [16, 18, 20, 22] or frame_keypoint_number in [15, 17, 19, 21]:
                hands_not_at_right_position = True
            if frame_keypoint_number in [11] or frame_keypoint_number in [12]:
                if left_shoulder_y is not None and right_shoulder_y is not None:
                    if left_shoulder_y < right_shoulder_y:
                        bent_right = True
                    elif left_shoulder_y > right_shoulder_y:
                        bent_left = True
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 0, 255), -1)

    if bent_left:
        label = keypoint_labels["BentLeft"]
    elif bent_right:
        label = keypoint_labels["BentRight"]
    elif leg_down:
        if hands_not_at_right_position:
            label = keypoint_labels["Idle"]
        else:
            label = keypoint_labels["LegDown"]
    elif hands_not_above:
        label = keypoint_labels["HandsNotAbove"]
    elif not hands_not_above and hands_not_at_right_position:
        label = keypoint_labels["HandsNotAtRightPosition"]
    else:
        label = keypoint_labels["Perfect"]

    label_lines = label.split('\n')
    for i, line in enumerate(label_lines):
        cv2.putText(frame_copy, line, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    output_video.write(frame_copy)
    cv2.imshow("Processed Video", frame_copy)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

input_video.release()
output_video.release()
cv2.destroyAllWindows()


masti

In [10]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose

# Create a function to calculate Euclidean distance between two points.
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Load the reference image.
reference_image = cv2.imread(r'dataset/dhruv_reference.jpg')

# Create a Pose object for reference image processing.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

# Process the reference image.
reference_results = pose.process(cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB))

# Retrieve the landmarks from the reference image.
reference_landmarks = []
if reference_results.pose_landmarks:
    for i, landmark in enumerate(reference_results.pose_landmarks.landmark):
        reference_landmarks.append((landmark.x, landmark.y, i))

# Define a dictionary to map keypoint numbers to body parts.
keypoint_labels = {
    0: "Nose",
    1: "Left Eye Inner",
    2: "Left Eye",
    3: "Left Eye Outer",
    4: "Right Eye Inner",
    5: "Right Eye",
    6: "Right Eye Outer",
    7: "Left Ear",
    8: "Right Ear",
    9: "Mouth Left",
    10: "Mouth Right",
    11: "Left Shoulder",
    12: "Right Shoulder",
    13: "Left Elbow",
    14: "Right Elbow",
    15: "Left Wrist",
    16: "Right Wrist",
    17: "Left Pinky",
    18: "Right Pinky",
    19: "Left Index",
    20: "Right Index",
    21: "Left Thumb",
    22: "Right Thumb",
    23: "Left Hip",
    24: "Right Hip",
    25: "Left Knee",
    26: "Right Knee",
    27: "Left Ankle",
    28: "Right Ankle",
    29: "Left Heel",
    30: "Right Heel",
    31: "Left Foot Index",
    32: "Right Foot Index"
}

# Print the keypoints and their corresponding body parts.
for landmark in reference_landmarks:
    x, y, keypoint_number = landmark
    body_part = keypoint_labels[keypoint_number]
    print(f"{body_part}: ({x}, {y})")


Nose: (0.5033059120178223, 0.15204253792762756)
Left Eye Inner: (0.5274533629417419, 0.13302761316299438)
Left Eye: (0.5428935289382935, 0.13307830691337585)
Left Eye Outer: (0.5525503754615784, 0.1336582899093628)
Right Eye Inner: (0.4888327121734619, 0.13416564464569092)
Right Eye: (0.47721368074417114, 0.13473135232925415)
Right Eye Outer: (0.4676058292388916, 0.1355954110622406)
Left Ear: (0.5682936906814575, 0.14232903718948364)
Right Ear: (0.4566885828971863, 0.1437881588935852)
Mouth Left: (0.5281562209129333, 0.17326602339744568)
Mouth Right: (0.48537951707839966, 0.1729699671268463)
Left Shoulder: (0.6558054685592651, 0.26218825578689575)
Right Shoulder: (0.37646496295928955, 0.25620216131210327)
Left Elbow: (0.7262104153633118, 0.3765597939491272)
Right Elbow: (0.2654138505458832, 0.3639928698539734)
Left Wrist: (0.5005074143409729, 0.37100186944007874)
Right Wrist: (0.42679333686828613, 0.3737330138683319)
Left Pinky: (0.4756017327308655, 0.3418465256690979)
Right Pinky: (0.

In [7]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

reference_image = cv2.imread(r'dataset/dhruv_reference.jpg')
# reference_image = cv2.imread(r'dataset/amit_reference.jpg')

pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

reference_results = pose.process(cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB))

reference_landmarks = []
if reference_results.pose_landmarks:
    for i, landmark in enumerate(reference_results.pose_landmarks.landmark):
        reference_landmarks.append((landmark.x, landmark.y, i))

input_video_path = r'dataset/videos/pcd.mp4'
# input_video_path = r'dataset/test.mp4'
# input_video = cv2.VideoCapture(0)
input_video = cv2.VideoCapture(input_video_path)
keypoint_labels = {
    "Perfect": "Perfect! Keep Going :)",
    "HandsNotAtRightPosition": "Hands not at right position",
    "HandsNotAt90": "Hands not at 90 degree",
    "LegsNotTriangle": "Legs not triangle",
    "Hands_legs_wrong": "Hands and leg both wrong",
    "LegDown": "Leg Down",
    "Idle": "Idle, please perform asana",
    "BentRight": "Bent right, \nstraighten yourself",
    "BentLeft": "Bent left, \nstraighten yourself",
    "BentForward": "Bent forward, straighten yourself"
}


while True:
    ret, frame = input_video.read()

    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_results = pose.process(frame_rgb)

    frame_landmarks = []
    if frame_results.pose_landmarks:
        for i, landmark in enumerate(frame_results.pose_landmarks.landmark):
            frame_landmarks.append((landmark.x, landmark.y, i))

    if len(frame_landmarks) > 0 and len(reference_landmarks) > 0:
        initial_offset = np.array(reference_landmarks[0][:2]) - np.array(frame_landmarks[0][:2])

    frame_copy = frame.copy()
    leg_down = False
    hands_not_at_right_position = False
    hands_not_at_90 = False
    bent_right = False
    bent_left = False
    bent_forward = False
    LegsNotTriangle = False

    right_shoulder_y = 0.0
    left_shoulder_y = 0.0


    for frame_landmark in frame_landmarks:
        frame_point = frame_landmark[:2]
        frame_keypoint_number = frame_landmark[2]

        if frame_keypoint_number == 11:
            left_shoulder_y = frame_landmark[1]
            
        if frame_keypoint_number == 12:
            right_shoulder_y = frame_landmark[1]
            
        # if frame_keypoint_number == 11:  # Left Shoulder
        #     left_shoulder_y = frame_point[1]
        # elif frame_keypoint_number == 12:  # Right Shoulder
        #     right_shoulder_y = frame_point[1]
        # elif frame_keypoint_number == 13:  # Left Elbow
        #     left_elbow_y = frame_point[1]
        # elif frame_keypoint_number == 14:  # Right Elbow
        #     right_elbow_y = frame_point[1]
        
        adjusted_left_shoulder_point = (reference_landmarks[11][0] - initial_offset[0], reference_landmarks[11][1] - initial_offset[1])
        adjusted_right_shoulder_point = (reference_landmarks[12][0] - initial_offset[0], reference_landmarks[12][1] - initial_offset[1])

        # Obtain y-coordinates of the adjusted reference frame for left and right shoulders
        adjusted_left_shoulder_y = adjusted_left_shoulder_point[1]
        adjusted_right_shoulder_y = adjusted_right_shoulder_point[1]

        adjusted_reference_point = (reference_landmarks[frame_keypoint_number][0] - initial_offset[0], reference_landmarks[frame_keypoint_number][1] - initial_offset[1])
        distance = calculate_distance(adjusted_reference_point, frame_point)
        threshold = 0.1

        
        if distance < threshold:
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 255, 0), -1)
        else:
            
            if frame_keypoint_number in [28, 30, 32] or frame_keypoint_number in [27, 29, 31] or frame_keypoint_number == 25 or frame_keypoint_number == 26:
                leg_down = True
            if frame_keypoint_number == 13 or frame_keypoint_number == 14:
                hands_not_at_90 = True
            if frame_keypoint_number in [16, 18, 20, 22] or frame_keypoint_number in [15, 17, 19, 21]:
                hands_not_at_right_position = True
            if frame_keypoint_number in [11] or frame_keypoint_number in [12]:
                # if left_shoulder_y is not None and right_shoulder_y is not None:
                    if left_shoulder_y < right_shoulder_y:
                        bent_right = True
                    elif left_shoulder_y > right_shoulder_y:
                        bent_left = True
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 0, 255), -1)

    if bent_left :
        label = keypoint_labels["BentLeft"]
    elif bent_right :
        label = keypoint_labels["BentRight"]
    elif leg_down:
        if hands_not_at_right_position:
            if not hands_not_at_90:
                label = keypoint_labels["Idle"]
            else:
                label = keypoint_labels["Hands_legs_wrong"]
        else:
            label = keypoint_labels["LegDown"]
    elif hands_not_at_90:
        label = keypoint_labels["HandsNotAt90"]
    elif hands_not_at_right_position:
        label = keypoint_labels["HandsNotAtRightPosition"]
    else:
        label = keypoint_labels["Perfect"]

    label_lines = label.split('\n')
    for i, line in enumerate(label_lines):
        cv2.putText(frame_copy, line, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Processed Video", frame_copy)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

input_video.release()
cv2.destroyAllWindows()
